Build a regression model.

My kernel constantly crashes no matter the dataset size when utilizing the statsmodel package, so I opted to utilize sklearn and scipy for regression and summary

In [56]:
# importing necessary modules
import pandas as pd
import numpy as np
import sqlite3
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from scipy import stats

In [57]:
'''
FETCHING DATA FROM DATABASE | LOADING IN CHUNKS DUE TO MEMORY ISSUES
'''
db_path = os.path.join('..', 'data', 'final_merged_dataset.db')

conn = sqlite3.connect(db_path)

query = 'SELECT * FROM final_merged'

final_merged_df = pd.read_sql_query(query, conn)

conn.close()

In [58]:
final_merged_df.head()

,id,alias,name,review_count,categories,rating,distance,business_hours,latitude,longitude,...,location.display_address,price,bike_station_lat,bike_station_lon,city,country,available_bikes,taken_bikes,total_bikes,percentage_bikes_taken
0,peGMNEAPUxOR4Trg15eVdA,le-bouquet-l-hospitalet-de-llobregat,Le Bouquet,3,"[{'alias': 'diners', 'title': 'Diners'}, {'ali...",4.3,125.860014,"[{'open': [{'is_overnight': False, 'start': '1...",41.347849,2.108678,...,"['Hotel Hesperia Tower', 'Gran Vía, 144', ""089...",None,41.40552,2.162255,Barcelona,ES,6,10,16,62.5
1,0_UULyMycCwHIqK0avhvxg,el-xiringuito-de-bellvitge-l-hospitalet-de-llo...,El Xiringuito de Bellvitge,3,"[{'alias': 'tapas', 'title': 'Tapas Bars'}]",4.7,260.177995,[],41.348536,2.111207,...,"['Rambla de la Marina, s/n', ""08901 L'Hospital...",€,41.40552,2.162255,Barcelona,ES,6,10,16,62.5
2,C65Z0hoJWndFHilQx6RxXw,bar-can-galan-hospitalet-de-llobregat,Bar Can Galan,5,"[{'alias': 'catalan', 'title': 'Catalan'}]",3.6,538.887813,[],41.350411,2.113510,...,"['Avinguda America, 37', '08907 Hospitalet de ...",€,41.40552,2.162255,Barcelona,ES,6,10,16,62.5
3,Ldwla3RbG0qq4UW4yVZm6w,la-marina-hospitalet-de-llobregat,La Marina,2,"[{'alias': 'spanish', 'title': 'Spanish'}, {'a...",4.0,852.140714,"[{'open': [{'is_overnight': False, 'start': '0...",41.339224,2.107411,...,"['Carrer de la Feixa Llarga, 47', '08907 Hospi...",None,41.40552,2.162255,Barcelona,ES,6,10,16,62.5
4,60GlRETCW6AXCS6d4gWQUA,da-fu-hau-hospitalet-de-llobregat,Da Fu Hau,2,"[{'alias': 'chinese', 'title': 'Chinese'}]",4.0,763.947944,[],41.353330,2.106090,...,"['Rambla Marina, 353', '08907 Hospitalet De Ll...",€,41.40552,2.162255,Barcelona,ES,6,10,16,62.5


In [59]:
'''
HAVE TO USE ONLY SMALL PORTION OF DATASET AS SIZE CAUSES CRASHES IN KERNEL
'''
randomized_final_df = final_merged_df.sample(frac=1, random_state=42)
randomized_final_df.head()

,id,alias,name,review_count,categories,rating,distance,business_hours,latitude,longitude,...,location.display_address,price,bike_station_lat,bike_station_lon,city,country,available_bikes,taken_bikes,total_bikes,percentage_bikes_taken
130020,BH0iza_rA57BLCXB8VgaDw,can-cuxant-barcelona,Can Cuxant,1,"[{'alias': 'spanish', 'title': 'Spanish'}, {'a...",1.0,299.214721,"[{'open': [{'is_overnight': False, 'start': '0...",41.428241,2.176145,...,"['Carrer Costa i Cuxart, 37', '08016 Barcelona...",€,41.405936,2.151798,Barcelona,ES,11,11,22,50.000000
221640,Ck-YsmCF_c-fApID2qQOfg,mas-pastissers-xocolaters-barcelona,Mas Pastissers Xocolaters,5,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",3.6,318.929993,"[{'open': [{'is_overnight': False, 'start': '0...",41.426270,2.172800,...,"['Passeig de Maragall, 269', '08032 Barcelona'...",€€€,41.394877,2.120539,Barcelona,ES,1,22,23,95.652174
24687,WOnKY-dvKu7WHwH9tCPczA,brasa-y-leña-hospitalet-de-llobregat,Brasa y Leña,3,"[{'alias': 'brazilian', 'title': 'Brazilian'}]",2.0,170.500334,[],41.358308,2.129159,...,"['Carrer Ciències, 54', '08908 Hospitalet de L...",None,41.400831,2.206645,Barcelona,ES,27,0,27,0.000000
56978,jjtoB00GRSXRgdpW9PnxuQ,masía-can-borrell-barcelona,Masía Can Borrell,5,"[{'alias': 'spanish', 'title': 'Spanish'}]",3.8,587.178692,[],41.462078,2.123660,...,"[""Carretera d'Horta a Cerdanyola BV-1415, km. ...",€€,41.397900,2.208845,Barcelona,ES,13,12,25,48.000000
161535,ju3v-KQRDiNPbIIO5thDlw,red-pizza-barcelona,Red Pizza,2,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,182.981048,"[{'open': [{'is_overnight': False, 'start': '1...",41.416520,2.196980,...,"['Rambla de Guipúscoa, 55', '08020 Barcelona',...",None,41.389462,2.131495,Barcelona,ES,1,26,27,96.296296


In [70]:
# data for the regression
X = sample_final_df[['rating']]
y = sample_final_df['available_bikes']

X_with_intercept = np.c_[np.ones(X.shape[0]), X]

In [71]:
X.shape

(197890, 1)

In [72]:
X.head()

,rating
0,1.0
1,3.6
2,2.0
3,3.8
4,4.0


In [73]:
y.head()

0    11
1     1
2    27
3    13
4     1
Name: available_bikes, dtype: int64

In [78]:
# fitting regression model
model = LinearRegression(fit_intercept=False).fit(X_with_intercept, y)
print(f"Intercept: {model.intercept_}")
print(f"Coefficients: {model.coef_}")

Intercept: 0.0
Coefficients: [11.4274681  -0.01396931]


Provide model output and an interpretation of the results. 

In [79]:
# predicitons
y_pred = model.predict(X_with_intercept)

# r_squared
r_squared = r2_score(y, y_pred)

# residuals
residuals = y - y_pred

In [83]:
# no. of observations and predictors
n = len(y)
p = X_with_intercept.shape[1]  

# mse
mse = mean_squared_error(y, y_pred)

# standard error of coefficients
sse = np.sum(residuals**2)
se_betas = np.sqrt(np.diagonal(mse * np.linalg.inv(X_with_intercept.T @ X_with_intercept)))

# t-statistics for the coefficients
t_stats = model.coef_ / se_betas

# p-values for the coefficients
p_values = [2 * (1 - stats.t.cdf(np.abs(t), df=n - p)) for t in t_stats]

In [84]:
summary_df = pd.DataFrame({
    'Coefficients': np.concatenate(([model.intercept_], model.coef_)),
    'Standard Errors': np.concatenate(([np.nan], se_betas)),  
    't-Values': np.concatenate(([np.nan], t_stats)),  
    'p-Values': np.concatenate(([np.nan], p_values))  
})

In [85]:
print(summary_df)
print(f'R-squared: {r_squared}')

   Coefficients  Standard Errors   t-Values  p-Values
0      0.000000              NaN        NaN       NaN
1     11.427468         0.119012  96.019683  0.000000
2     -0.013969         0.028340  -0.492915  0.622073
R-squared: 1.2277792450499092e-06


I searched on how to get most of these values due to infamiliarity with packages

With an r-squared being so close to 0, it signifies that ratings of nearby POIs is not a good indicator for bike availability at nearby bike stations.

A p-value of 0.6 indicates very little evidence in a relationship between the two variables. Typically, if there were to be a relationship, the p-value would be around 0.05.